<a href="https://colab.research.google.com/github/aman-welzin/credzin/blob/main/pycode/src/rags/recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q -U langchain langchain-community langchain_experimental gradio pymupdf python-docx ollama langchain-huggingface langchain-neo4j lxml-html-clean pillow langgraph yfiles_jupyter_graphs neo4j langchain-neo4j langchain-ollama

# Install Ollama and CUDA drivers
import os
!nvidia-smi
!curl https://ollama.ai/install.sh | sh
!echo 'debconf debconf/frontend select Noninteractive' | sudo debconf-set-selections
!sudo apt-get update && sudo apt-get install -y cuda-drivers
# Set LD_LIBRARY_PATH so the system NVIDIA library
os.environ.update({'LD_LIBRARY_PATH': '/usr/lib64-nvidia'})

!nohup ollama serve &
!ollama ps
# !ollama pull mistral-small3.1
!ollama pull llama3.2
!ollama list

In [ ]:
!pip install --upgrade --force-reinstall numpy pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 114.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 109.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.9/229.9 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.2/509.2 kB 38.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.8/347.8 kB 29.4 MB/s eta 0:00:00
  Attempting uninstall: pytz
    Found existing installation: pytz 2025.2
    Uninstalling pytz-2025.2:
      Successfully uninstalled pytz-2025.2
  Attempting uninstall: tzdata
    Found existing installation: tzdata 2025.2
    Uninstalling tzdata-2025.2:
      Successfully uninstalled tzdata-2025.2
  Attempting uninstall: six
    Found existing installation: six 1.17.0
    Uninstalling six-1.17.0:
      Successfully uninstalled six

In [ ]:
import io
import os, uuid
from datetime import datetime
import pandas as pd
from tqdm.notebook import tqdm
import json
import re
import urllib
import pprint
from IPython import embed
import matplotlib.font_manager
import matplotlib as mpl
from yfiles_jupyter_graphs import GraphWidget
import codecs
import base64
import PIL
from PIL import Image, ImageFont, ImageDraw, ImageColor
import textwrap
from IPython.display import Image, display
from neo4j import GraphDatabase
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain_community.llms import Ollama
from langchain_core.prompts import ChatPromptTemplate
from langchain import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import AIMessage, HumanMessage
from langchain.schema import Document
from langchain.docstore import InMemoryDocstore
from langchain.retrievers import ParentDocumentRetriever
from google.colab import userdata
from langgraph.graph.message import add_messages
from langgraph.graph import END, StateGraph, START
from langchain.graphs import Neo4jGraph
from typing import List
from typing_extensions import TypedDict, Annotated
from pydantic import BaseModel, Field
from neo4j_graphrag.retrievers import HybridRetriever
from langchain.chains import GraphCypherQAChain

import time
import threading
from concurrent.futures import ThreadPoolExecutor, as_completed
from google.colab import drive
import warnings
warnings.filterwarnings('ignore')

In [ ]:
!nohup ollama serve &

nohup: appending output to 'nohup.out'


In [ ]:
EMBEDDING_MODEL = "sentence-transformers/static-retrieval-mrl-en-v1"
LLAMA_MODEL = "llama3.2"

In [ ]:
NEO4J_URI="neo4j+s://c64a3769.databases.neo4j.io"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="mzP-L3pD-nGTc3hFpli2T7DsNbFA7jbp5bWHno1NSH4"
AURA_INSTANCEID="c64a3769"
AURA_INSTANCENAME="Instance03"

CSV_PATH = '/content/cc_feats.csv'

AUTH=(NEO4J_USERNAME, NEO4J_PASSWORD)
with GraphDatabase.driver(NEO4J_URI, auth=AUTH) as driver:
    driver.verify_connectivity()

ONLY RUN THIS CELL WHEN NEW CARDS ARE ADDED TO THE CSV

In [ ]:
import csv
from neo4j import GraphDatabase, basic_auth
from langchain.chains import GraphCypherQAChain


def clean_list_field(field: str, delimiters: tuple = (";", ","), strip_chars: str = '[]'):
    """
    Remove surrounding characters and split by delimiters.
    Returns list of stripped strings.
    """
    if not field:
        return []
    # remove bracket-like characters
    for ch in strip_chars:
        field = field.replace(ch, '')
    # split by provided delimiters
    for delim in delimiters[1:]:
        field = field.replace(delim, delimiters[0])
    items = [item.strip() for item in field.split(delimiters[0]) if item.strip()]
    return items


def to_float(value: str):
    """
    Remove currency symbols and converts to float
    """
    if not value:
        return 0.0
    v = value.replace('₹', '').replace('%', '').replace(',', '').strip()
    try:
        return float(v)
    except ValueError:
        return 0.0


# Neo4j import logic

def create_constraints(tx):
    tx.run("CREATE CONSTRAINT bank_name_unique IF NOT EXISTS FOR (b:Bank) REQUIRE b.name IS UNIQUE")
    tx.run("CREATE CONSTRAINT card_name_unique IF NOT EXISTS FOR (c:Card) REQUIRE c.name IS UNIQUE")
    tx.run("CREATE CONSTRAINT category_name_unique IF NOT EXISTS FOR (cat:Category) REQUIRE cat.name IS UNIQUE")
    tx.run("CREATE CONSTRAINT feature_name_unique IF NOT EXISTS FOR (f:Feature) REQUIRE f.name IS UNIQUE")
    tx.run("CREATE CONSTRAINT reward_desc_unique IF NOT EXISTS FOR (r:Reward) REQUIRE r.description IS UNIQUE")




def import_row(tx, row):
    # Merge Bank and Card
    tx.run(
        """
        MERGE (b:Bank {name: $bank_name})
        MERGE (c:Card {name: $card_name})
        SET c.joining_fee       = $joining_fee,
            c.annual_fee        = $annual_fee,
            c.annual_fee_waiver = $annual_fee_waiver,
            c.add_on_card_fee   = $add_on_card_fee,
            c.interest_rate_pa  = $interest_rate_pa,
            c.card_type         = $card_type,
            c.usp               = $usp,
            c.image_url         = $image_url,
            c.apply_now_link    = $apply_now_link,
            c.know_more_link    = $know_more_link
        MERGE (b)-[:ISSUES]->(c)
        """,
        row)

    # Categories
    for cat in clean_list_field(row['card_category'], delimiters=(',',), strip_chars='[]'):
        tx.run(
            """
            MERGE (cat:Category {name: $cat})
            WITH cat
            MATCH (c:Card {name: $card_name})
            MERGE (c)-[:BELONGS_TO]->(cat)
            """,
            {"cat": cat, "card_name": row['card_name']}
        )

    # Features
    for feat in clean_list_field(row.get('features_clean', ''), delimiters=(',',), strip_chars=''):
        tx.run(
            """
            MERGE (f:Feature {name: $feat})
            WITH f
            MATCH (c:Card {name: $card_name})
            MERGE (c)-[:HAS_FEATURE]->(f)
            """,
            {"feat": feat, "card_name": row['card_name']}
        )

    # Rewards
    for rew in clean_list_field(row.get('rewards_clean', ''), delimiters=(';;',), strip_chars=''):
        tx.run(
            """
            MERGE (r:Reward {description: $rew})
            WITH r
            MATCH (c:Card {name: $card_name})
            MERGE (c)-[:OFFERS_REWARD]->(r)
            """,
            {"rew": rew, "card_name": row['card_name']}
        )


def main():
    driver = GraphDatabase.driver(NEO4J_URI, auth=AUTH)
    with driver.session() as session:
        # 1) Create constraints
        session.write_transaction(create_constraints)

        # 2) Read CSV and import rows
        with open(CSV_PATH, newline='', encoding='utf-8') as csvfile:
            reader = csv.DictReader(csvfile)
            for raw in reader:
                # Prepare row params
                row = {
                    'bank_name':         raw.get('bank_name'),
                    'card_name':         raw.get('card_name'),
                    'joining_fee':       to_float(raw.get('joining_fee', '')),
                    'annual_fee':        to_float(raw.get('annual_fee', '')),
                    'annual_fee_waiver': raw.get('annual_fee_waiver', ''),
                    'add_on_card_fee':   to_float(raw.get('add_on_card_fee', '')),
                    'interest_rate_pa':  to_float(raw.get('interest_rate_pa', '')),
                    'card_type':         raw.get('card_type', ''),
                    'usp':               raw.get('card_usp', ''),
                    'image_url':         raw.get('image_url', ''),
                    'apply_now_link':    raw.get('apply_now_link', ''),
                    'know_more_link':    raw.get('know_more_link', ''),
                    'card_category':     raw.get('card_category', ''),
                    'features_clean':    raw.get('features_clean', ''),
                    'rewards_clean':     raw.get('rewards_clean', '')
                }
                session.write_transaction(import_row, row)

    driver.close()


main()


In [ ]:
# # 1) Instantiate your Neo4jGraph & Chain once
# _graph = Neo4jGraph(
#     url="neo4j+s://c64a3769.databases.neo4j.io",
#     username="neo4j",
#     password="mzP-L3pD-nGTc3hFpli2T7DsNbFA7jbp5bWHno1NSH4"
# )

# _prompt = PromptTemplate(
#     input_variables=["query"],
#     template="""Task: Generate Cypher statement to query a Neo4j database.
# Instructions:
# - Use only the provided relationship types and properties.
# - Use toLower(...).contains("...") when searching for values.
# - Prefer searching in Feature.name or Reward.description for offers/discounts.
# - When the query asks for a credit card's details, always MATCH both the Card and its issuing Bank.

# Schema:
# (:Bank {{name}})
# (:Card {{name, joining_fee, annual_fee, annual_fee_waiver, add_on_card_fee, interest_rate_pa, card_type, usp, image_url, apply_now_link, know_more_link}})
# (:Feature {{name}})
# (:Reward {{description}})
# (:Category {{name}})
# Relationships:
# (Bank)-[:ISSUES]->(Card)
# (Card)-[:HAS_FEATURE]->(Feature)
# (Card)-[:OFFERS_REWARD]->(Reward)
# (Card)-[:BELONGS_TO]->(Category)

# Example:
# # Show details for a given credit card
# MATCH (b:Bank)-[:ISSUES]->(c:Card {{name: "Axis Bank AURACredit Card"}})
# OPTIONAL MATCH (c)-[:HAS_FEATURE]->(f:Feature)
# OPTIONAL MATCH (c)-[:OFFERS_REWARD]->(r:Reward)
# OPTIONAL MATCH (c)-[:BELONGS_TO]->(cat:Category)
# RETURN
#   c.name       AS CardName,
#   c.card_type  AS CardType,
#   c.usp        AS USP,
#   c.joining_fee        AS JoiningFee,
#   c.annual_fee         AS AnnualFee,
#   c.annual_fee_waiver  AS AnnualFeeWaiver,
#   c.interest_rate_pa   AS InterestRate,
#   c.image_url          AS ImageURL,
#   c.apply_now_link     AS ApplyNow,
#   c.know_more_link     AS KnowMore,
#   b.name               AS Issuer,
#   collect(DISTINCT f.name)         AS Features,
#   collect(DISTINCT r.description)  AS Rewards,
#   collect(DISTINCT cat.name)       AS Categories

# The question is:
# {query}
# """
# )

# _chain = GraphCypherQAChain.from_llm(
#     llm=Ollama(model="llama3.2"),
#     graph=_graph,
#     verbose=False,
#     cypher_prompt=_prompt,
#     return_direct=True,
#     allow_dangerous_requests=True,
# )

# def get_card_details(card_name: str, retries=1) -> dict:
#     query = f"Show all the details of the credit card named {card_name}"
#     for _ in range(retries + 1):
#         try:
#             return _chain.invoke({"query": query})
#         except Exception as e:
#             if "Variable `b` not defined" in str(e) and retries > 0:
#                 print(f"Retrying query for {card_name} due to missing `b` match...")
#                 continue
#             raise


In [ ]:
# 1) Driver setup (only once)
_URI      = "neo4j+s://c64a3769.databases.neo4j.io"
_USER     = "neo4j"
_PASSWORD = "mzP-L3pD-nGTc3hFpli2T7DsNbFA7jbp5bWHno1NSH4"

_driver = GraphDatabase.driver(_URI, auth=(_USER, _PASSWORD))

# 2) Low-level Cypher executor
def _run_cypher(cypher: str, params: dict) -> list[dict]:
    with _driver.session() as session:
        result = session.run(cypher, params)
        return [record.data() for record in result]

# 3) Your simplified get_card_details (no retries)
def get_card_details(card_name: str) -> dict:
    """
    Show all the details of the credit card named `card_name`.
    Returns a dict in the form:
      {
        "query": "Show all the details of the credit card named …",
        "result": [ { CardName:…, Features:[…], … }, … ]
      }
    """
    cypher = """
    MATCH (b:Bank)-[:ISSUES]->(c:Card {name: $card_name})
    OPTIONAL MATCH (c)-[:HAS_FEATURE]->(f:Feature)
    OPTIONAL MATCH (c)-[:OFFERS_REWARD]->(r:Reward)
    OPTIONAL MATCH (c)-[:BELONGS_TO]->(cat:Category)
    RETURN
      c.name               AS CardName,
      c.card_type          AS CardType,
      c.usp                AS USP,
      c.joining_fee        AS JoiningFee,
      c.annual_fee         AS AnnualFee,
      c.annual_fee_waiver  AS AnnualFeeWaiver,
      c.interest_rate_pa   AS InterestRate,
      c.image_url          AS ImageURL,
      c.apply_now_link     AS ApplyNow,
      c.know_more_link     AS KnowMore,
      b.name               AS Issuer,
      collect(DISTINCT f.name)         AS Features,
      collect(DISTINCT r.description)  AS Rewards,
      collect(DISTINCT cat.name)       AS Categories
    """
    rows = _run_cypher(cypher, {"card_name": card_name})
    return {
        "query": f"Show all the details of the credit card named {card_name}",
        "result": rows
    }

In [ ]:
details = get_card_details("Axis Bank AURACredit Card")
print(details)

{'query': 'Show all the details of the credit card named Axis Bank AURACredit Card', 'result': [{'CardName': 'Axis Bank AURACredit Card', 'CardType': '', 'USP': '', 'JoiningFee': 0.0, 'AnnualFee': 0.0, 'AnnualFeeWaiver': '', 'InterestRate': 0.0, 'ImageURL': 'https://www.axisbank.com/images/default-source/revamp_new/cards/cc-new/aura-credit-card-facia.jpg', 'ApplyNow': 'https://web.axisbank.co.in/DigitalChannel/WebForm/?index6&utm_content=cclisting&utm_campaign=cciocl&utm_source=website&axisreferralcode=iocllisting', 'KnowMore': 'https://www.axisbank.com/retail/cards/credit-card/axis-bank-aura-credit-card', 'Issuer': 'Axis', 'Features': ['Get welcome decathlon OMNI Gift Card worth INR 750', 'Earn 2 points per spend of INR 200 and 5x reward points on Insurance', 'Get 48 online video doctor consultations free per year', 'Get 48 interactive fitness video training sessions free per year', 'Get discount upto INR 500 on health checkup'], 'Rewards': ['[]'], 'Categories': []}]}


In [ ]:
details = get_card_details("Gold Card")
print(details)

{'query': 'Show all the details of the credit card named Gold Card', 'result': [{'CardName': 'Gold Card', 'CardType': 'Premium Credit Card', 'USP': '4X points on restaurants & at U.S. supermarkets, $120 Dining Credit, $120 Uber Cash, $100 Resy Credit, and more.', 'JoiningFee': 0.0, 'AnnualFee': 0.0, 'AnnualFeeWaiver': '', 'InterestRate': 0.0, 'ImageURL': 'https://icm.aexp-static.com/Internet/internationalcardshop/en_in/images/cards/Gold_Card.png', 'ApplyNow': 'https://www.americanexpress.com/en-us/account/get-started/gold/', 'KnowMore': 'https://www.americanexpress.com/us/credit-cards/card/gold-card/', 'Issuer': 'Amex', 'Features': ['and more.', 'Uber Cash', 'Travel perks', 'Dining credits', 'Resy Credit'], 'Rewards': ['Earn up to 4x points on restaurants & at U.S. supermarkets'], 'Categories': ['Dining and Travel']}]}


In [ ]:
details = get_card_details("SBI Card PRIME")
print(details)

{'query': 'Show all the details of the credit card named SBI Card PRIME', 'result': [{'CardName': 'SBI Card PRIME', 'CardType': '', 'USP': '', 'JoiningFee': 0.0, 'AnnualFee': 0.0, 'AnnualFeeWaiver': '', 'InterestRate': 0.0, 'ImageURL': 'https://www.sbicard.com/sbi-card-en/assets/media/images/personal/credit-cards/network-card-images/Prime_349x218-px-Front.png', 'ApplyNow': 'https://www.sbicard.com/en/eapply/eapplyform.page?path=personal/credit-cards/rewards/sbi-card-prime.dcr', 'KnowMore': 'https://www.sbicard.com/en/personal/credit-cards/rewards/sbi-card-prime.page', 'Issuer': 'SBI', 'Features': ['000', '000 on joining', 'Welcome e-Gift Voucher worthRs. 3', 'Spend linked Gift Vouchers worthRs. 11', '10 Reward Points perRs. 100 spent on Dining', 'Groceries', 'Departmental stores and Movies', 'Complimentary International and Domestic Airport Lounge access'], 'Rewards': ["{'Terms and Conditions': ['For detailed Terms and Conditions and Usage Directions, click\\xa0here'], 'Mastercard/Visa

In [ ]:
details = get_card_details("Wellness Plus Credit Card")
print(details)

{'query': 'Show all the details of the credit card named Wellness Plus Credit Card', 'result': [{'CardName': 'Wellness Plus Credit Card', 'CardType': '', 'USP': 'Holistic health, self-care and wellness of its users', 'JoiningFee': 1499.0, 'AnnualFee': 0.0, 'AnnualFeeWaiver': '', 'InterestRate': 0.0, 'ImageURL': '', 'ApplyNow': 'https://www.yesbank.in/personal-banking/yes-individual/cards/credit-cards/wellness-plus-cards', 'KnowMore': 'https://www.yesbank.in/personal-banking/yes-individual/cards/credit-cards/wellness-plus-cards', 'Issuer': 'Yes', 'Features': ['Annual health check-ups', 'round the clock doctor or counsellor helpline', 'in-studio or home-based fitness sessions'], 'Rewards': ['30 Reward Points per Rs. 200 spent on Chemist/Pharmaceutical stores'], 'Categories': []}]}


In [ ]:
from langchain_community.chat_models import ChatOllama
from langchain_core.messages import HumanMessage
import ast
import re

# Initialize the Ollama LLM
llm_summarizer = ChatOllama(
    base_url="http://localhost:11434",
    temperature=0,
    model="llama3.2",
)

def extract_list_from_string(text):
    """Extracts the first valid Python-style list from LLM output using regex."""
    match = re.search(r'\[.*\]', text, re.DOTALL)
    if match:
        try:
            return ast.literal_eval(match.group(0))
        except Exception as e:
            print("List parsing error:", e)
            return []
    return []

def extract_card_names(query: str) -> list:
    prompt = f"""
You are an assistant that extracts **credit card names** from user queries.
Only return a **Python list of strings** — the card names mentioned in the query.

Example:
Input: Compare Axis Bank AURACredit Card with HDFC MoneyBack Card and SBI Premier Credit Card
Output: ["Axis Bank AURACredit Card", "HDFC MoneyBack Card", "SBI Premier Credit Card"]


Extract only the list of credit card names mentioned in the input.
Only return a Python list like: ["Card A", "Card B"]

Do not return code, explanation, or anything else.

Input: {query}
Output:
"""

    try:
        response = llm_summarizer.invoke([HumanMessage(content=prompt)])
        output = response.content.strip()
        print("LLM Output:", output)  # For debugging
        return extract_list_from_string(output)
    except Exception as e:
        print("Error:", e)
        return []

# Example usage
query = "Let's say Wellness Plus Credit Card, Gold Card, SBI Card PRIME Card, Elite+ Credit Card and BYOC Credit Card which one is the best"
print(extract_card_names(query))


In [ ]:
def process_user_query(query: str) -> list[dict]:
    """
    Given a user query, extracts card names and returns details for all those cards.

    :param query: user input string
    :return: list of card details (one per card)
    """
    card_names = extract_card_names(query)
    print("Extracted Card Names:", card_names)

    all_details = []
    for card in card_names:
        try:
            details = get_card_details(card)
            all_details.append(details)
        except Exception as e:
            print(f"Failed to get details for {card}: {e}")
            all_details.append({"CardName": card, "error": str(e)})
    return all_details

# Example usage
query = "Let's say I have Wellness Plus Credit Card, Gold Card, SBI Card PRIME, Elite+ Credit Card and BYOC Credit Card which one is the best"
result = process_user_query(query)
for card_info in result:
    print(card_info)

LLM Output: ["Wellness Plus Credit Card", "Gold Card", "SBI Card PRIME", "Elite+ Credit Card", "BYOC Credit Card"]
Extracted Card Names: ['Wellness Plus Credit Card', 'Gold Card', 'SBI Card PRIME', 'Elite+ Credit Card', 'BYOC Credit Card']
Retrying query for Elite+ Credit Card due to missing `b` match...
{'query': 'Show all the details of the credit card named Wellness Plus Credit Card', 'result': [{'CardName': 'Wellness Plus Credit Card', 'CardType': '', 'USP': 'Holistic health, self-care and wellness of its users', 'JoiningFee': 1499.0, 'AnnualFee': 0.0, 'AnnualFeeWaiver': '', 'InterestRate': 0.0, 'ImageURL': '', 'ApplyNow': 'https://www.yesbank.in/personal-banking/yes-individual/cards/credit-cards/wellness-plus-cards', 'KnowMore': 'https://www.yesbank.in/personal-banking/yes-individual/cards/credit-cards/wellness-plus-cards', 'Issuer': 'Yes', 'Features': ['Annual health check-ups', 'round the clock doctor or counsellor helpline', 'in-studio or home-based fitness sessions'], 'Rewards

In [ ]:
from langchain_community.chat_models import ChatOllama
from langchain_core.messages import HumanMessage

# LLM setup
llm = ChatOllama(base_url="http://localhost:11434", temperature=0, model="llama3.2")

def summarize_card_info(card_data: dict) -> str:
    prompt = f"""
You are a helpful assistant. Given the following raw credit card data in JSON-like format,
summarize the **key benefits**, **main features**, and **rewards** this card provides.

Also, mention any other notable or special details.

Card Data:
{card_data}

Now summarize clearly:
- Key Benefits:
- Key Features:
- Key Rewards:
- Other Notable Info:
"""
    response = llm.invoke([HumanMessage(content=prompt)])
    return response.content.strip()


card_summaries = {}


for card in result:
    if "result" in card and isinstance(card["result"], list) and len(card["result"]) > 0:
        card_data = card["result"][0]  # Use first entry
        card_name = card_data.get("CardName", "Unknown Card")
        summary = summarize_card_info(card_data)
        card_summaries[card_name] = summary  # Save it for later
        print(f"\nSummary for {card_name}:\n")
        print(summary)



Summary for Wellness Plus Credit Card:

Based on the provided credit card data, here's a summary of the key benefits, main features, and rewards:

**Key Benefits:**

1. Holistic health and self-care support through annual health check-ups.
2. Access to round-the-clock doctor or counselor helpline for medical assistance.
3. In-studio or home-based fitness sessions for physical well-being.

**Main Features:**

1. Annual health check-ups.
2. Round-the-clock doctor or counselor helpline.
3. In-studio or home-based fitness sessions.
4. Holistic health and self-care support.

**Key Rewards:**

1. 30 Reward Points per Rs. 200 spent on Chemist/Pharmaceutical stores.

**Other Notable Info:**

1. The card has a joining fee of Rs. 1499.00.
2. There is no annual fee for this card.
3. The issuer of the card is Yes Bank.
4. More information about the card can be found on the official website: https://www.yesbank.in/personal-banking/yes-individual/cards/credit-cards/wellness-plus-cards
5. To apply f

In [ ]:
def analyze_card_summaries(all_summaries: dict) -> str:
    # 1) Flatten summaries into one string
    summaries_text = "\n\n".join(
        f"Summary for {card}:\n{summary}"
        for card, summary in all_summaries.items()
    )

    prompt = f"""
You are a rewards analysis assistant.

STEP 1 – CARD ANALYSIS:
For each credit card summary below, extract and list the types of benefits or rewards it provides, using these **standard** categories if they appear:
  welcome_points, milestone_rewards, bonus_points, cashback_offer,
  voucher_offer, travel_rewards, fuel_rewards, movie_rewards.

If a benefit doesn’t exactly match one of those, infer an appropriate tag from this **extended** list:
  health benefits, movie discounts, shopping benefits, lounge access,
  fuel benefits, dining offers, electronic offers, customizable features, etc.

In the **Reason** section, for each category you assign, quote the exact text from the summary that justifies it.

---

STEP 2 – FINDING MISSING CATEGORIES:
At the end, propose **exactly two** categories the user does **not** already have—i.e. do **not** repeat any category you assigned above.
- You may suggest a **sub‐type** of an existing category (e.g. `flight_travel_rewards`) but only if you clearly distinguish it (and justify why it isn’t covered by the generic tag).
- Do **not** list `travel_rewards` or `shopping_benefits` again since they already exist; instead find truly new angles.

Return your answer in this format:

---
**Card: <Card Name>**
Categories: <comma-separated list>
Reason:
  - <Category1>: “<quote from summary>”
  - <Category2>: “<quote from summary>”
  …

… (repeat for each card) …

**Missing Categories:**
- <CategoryX>: <1–2 line justification why this angle isn’t covered>
- <CategoryY>: <1–2 line justification why this angle isn’t covered>

Here are the summaries:
{summaries_text}
"""

    response = llm.invoke([HumanMessage(content=prompt)])
    return response.content.strip()


In [ ]:
analysis = analyze_card_summaries(card_summaries)
print(analysis)

**Card: Wellness Plus Credit Card**
Categories: health_benefits, wellness_programs
Reason:
  - health_benefits: “Holistic health and self-care support through annual health check-ups.”
  - wellness_programs: “Holistic health and self-care support.”

**Card: Gold Card**
Categories: travel_rewards, dining_offers
Reason:
  - travel_rewards: “Earn up to 4x points on restaurants & at U.S. supermarkets” (travel-related benefits)
  - dining_offers: “$120 Dining Credit” (direct benefit for dining)

**Card: SBI Card PRIME**
Categories: travel_rewards, fuel_rewards, dining_offers
Reason:
  - travel_rewards: “Complimentary international and domestic airport lounge access” (airport lounge access is a form of travel-related benefit)
  - fuel_rewards: “Savings of up to 10% on Hertz car rental and up to 35% on Avis Car Rentals” (fuel-related benefits)
  - dining_offers: “Up to 15% discount on best available room rates at Lalit Hotels” (direct benefit for dining)

**Card: Elite+ Credit Card**
Categori

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import GraphCypherQAChain
from langchain_community.llms import Ollama
import json

schema = """
(:Bank {name})
(:Card {name, joining_fee, annual_fee, annual_fee_waiver, add_on_card_fee, interest_rate_pa, card_type, usp, image_url, apply_now_link, know_more_link})
(:Feature {name})
(:Reward {description})
(:Category {name})

Relationships:
(Bank)-[:ISSUES]->(Card)
(Card)-[:HAS_FEATURE]->(Feature)
(Card)-[:OFFERS_REWARD]->(Reward)
(Card)-[:BELONGS_TO]->(Category)
"""

CYPHER_GENERATION_TEMPLATE = """Task: Generate Cypher statement to query a Neo4j database.
Instructions:
- Use only the provided relationship types and properties in the schema.
- Use `toLower(...).contains("...")` when searching for values.
- Prefer searching in Feature.name or Reward.description when the user asks for offers/discounts.

Schema:
{schema}

Examples:
# Which credit cards offer discounts on Decathlon?
MATCH (c:Card)-[:HAS_FEATURE]->(f:Feature)
WHERE toLower(f.name) CONTAINS "decathlon"
RETURN c.name AS CardName

# List credit cards that offer movie ticket benefits
MATCH (c:Card)-[:HAS_FEATURE]->(f:Feature)
WHERE toLower(f.name) CONTAINS "movie"
RETURN c.name AS CardName

# Credit cards with BookMyShow offers
MATCH (c:Card)-[:HAS_FEATURE]->(f:Feature)
WHERE toLower(f.name) CONTAINS "bookmyshow"
RETURN c.name AS CardName

The question is:
{question}
"""

CYPHER_GENERATION_PROMPT = PromptTemplate(
    input_variables=["schema", "question"],
    template=CYPHER_GENERATION_TEMPLATE
)

chain = GraphCypherQAChain.from_llm(
    llm=Ollama(model="llama3.2"),
    graph=_graph,
    verbose=True,
    cypher_prompt=CYPHER_GENERATION_PROMPT,
    return_direct=True,
    allow_dangerous_requests=True,
)

def get_card_details_safe(card_name: str) -> dict:
    """Fetch full card details, catch and return any errors."""
    try:
        return get_card_details(card_name)
    except Exception as e:
        return {"CardName": card_name, "error": str(e)}

# 🔁 Now use your new missing categories:
missing_categories = ["flight_travel_rewards", "digital_offers"]

cards_info: dict[str, list[dict]] = {}

for category in missing_categories:
    # Build a human-readable question, e.g. "flight travel rewards"
    human_q = category.replace("_", " ")
    question = f"Suggest all the credit cards that give {human_q}"

    response = chain.invoke({"query": question})

    # Extract and dedupe card names
    names = [row["CardName"] for row in response.get("result", [])]
    unique_names = list(dict.fromkeys(names))

    # Fetch each card's details
    details_list = [get_card_details_safe(name) for name in unique_names]
    cards_info[category] = details_list

# Save to disk or inspect
with open("cards_info.json", "w") as f:
    json.dump(cards_info, f, indent=2)

from pprint import pprint
pprint(cards_info)




> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (c:Card)-[:HAS_FEATURE]->(f:Feature)
WHERE f.name CONTAINS "flight" OR f.name CONTAINS "air travel"
RETURN c.name AS CardName


> Finished chain.
Retrying query for SpiceJet Axis Bank Voyage BlackCredit Card due to missing `b` match...


> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (c:Card)-[:HAS_FEATURE]->(f:Feature)
WHERE toLower(f.name) CONTAINS "digital"
RETURN c.name AS CardName, f.name AS FeatureName


> Finished chain.


In [ ]:
from langchain_community.chat_models import ChatOllama
from langchain_core.messages import HumanMessage

# ─── LLM SETUP ────────────────────────────────────────────────────────────────
llm = ChatOllama(
    base_url="http://localhost:11434",
    temperature=0,
    model="llama3.2",
)

# ─── CARDS THE USER ALREADY HAS ───────────────────────────────────────────────
user_cards = [
    "Wellness Plus Credit Card",
    "Gold Card",
    "SBI Card PRIME",
    "Elite+ Credit Card",
    "BYOC Credit Card"
]

def summarize_card_info(card_data: dict) -> str:
    prompt = f"""
You are a helpful assistant. Given the following raw credit card data in JSON-like format,
summarize the **key benefits**, **main features**, and **rewards** this card provides.

Also, mention any other notable or special details.

Card Data:
{card_data}

Now summarize clearly:
- Key Benefits:
- Key Features:
- Key Rewards:
- Other Notable Info:
"""
    resp = llm.invoke([HumanMessage(content=prompt)])
    return resp.content.strip()

def choose_best_card(category: str,
                     summaries: dict[str,str],
                     user_cards: list[str]) -> str:
    """
    - category: e.g. "shopping_discounts"
    - summaries: map of card name -> its four-line summary
    - user_cards: list of cards the user already holds
    """
    existing = ", ".join(user_cards)
    bullet_list = "\n".join(
        f"{i+1}. **{name}**:\n{summaries[name]}"
        for i,name in enumerate(summaries)
    )

    prompt = f"""
You are an expert financial advisor.

The user currently holds these cards: {existing}.

Below are summaries of credit cards that offer {category.replace('_',' ')}:

{bullet_list}

Task:
1. Identify which of these would be the **Best Card** for {category.replace('_',' ')}.
2. In your **Reason**, follow this structure:
   - Mention that the user does **not** currently have any {category.replace('_',' ')} rewards.
   - List the **specific rewards** from the recommended card that cover that category.
   - Call out **any additional facilities or benefits** the card provides beyond that category.

Answer in this format:

- **Best Card**: <Card Name>
- **Reason**: Based on your existing cards (<existing>), you lack any {category.replace('_',' ')}.
  The **<Best Card>** provides <list of category-specific rewards> and also offers <list of additional facilities>.
"""
    resp = llm.invoke([HumanMessage(content=prompt)])
    return resp.content.strip()


final_recommendations: dict[str, str] = {}

for category, details_list in cards_info.items():
    # 1) Build summaries for each candidate
    card_summaries: dict[str,str] = {}
    for entry in details_list:
        if not entry or "result" not in entry or not entry["result"]:
            continue
        card_data = entry["result"][0]
        name = card_data.get("CardName", "Unknown")
        card_summaries[name] = summarize_card_info(card_data)

    # 2) Pick the best card with a tailored reason
    recommendation = choose_best_card(category, card_summaries, user_cards)
    final_recommendations[category] = recommendation

    print(f"\n=== Recommendation for {category} ===\n{recommendation}\n")

# final_recommendations now holds the pick + a reason


In [ ]:
import re
from pprint import pprint

# 2) Parse it into a dict: { card_name: [cat1, cat2, …], … }
user_card_categories = {}
# find each “**Card: …**” and the next “Categories: …” line
for match in re.finditer(
    r"\*\*Card: (?P<card>.*?)\*\*.*?Categories: (?P<cats>.*?)\n",
    analysis,
    re.DOTALL
):
    card = match.group("card").strip()
    cats = [c.strip() for c in match.group("cats").split(",")]
    user_card_categories[card] = cats

print("Parsed user cards → categories:")
pprint(user_card_categories)


# 3) Now run your existing GraphCypherQAChain loop for each card:
cards_info_by_user_card = {}

for card_name, categories in user_card_categories.items():
    human_q = " and ".join(categories)
    question = f"Suggest all the credit cards that give {human_q}"

    # generate & run the Cypher
    response = chain.invoke({"query": question})

    # dedupe
    names = [row["CardName"] for row in response.get("result", [])]
    unique_names = list(dict.fromkeys(names))

    # fetch details
    details_list = [get_card_details_safe(n) for n in unique_names]
    cards_info_by_user_card[card_name] = details_list

# 4) Inspect
from pprint import pprint
pprint(cards_info_by_user_card)


In [ ]:
final_recommendations = {}

for card_name, card_details in cards_info_by_user_card.items():
    # Summarize all similar cards except the current one
    card_summaries = {}
    for detail in card_details:
        if not detail or "result" not in detail or not detail["result"]:
            continue
        card_data = detail["result"][0]
        name = card_data.get("CardName", "Unknown")
        if name == card_name:
            continue  # skip self
        card_summaries[name] = summarize_card_info(card_data)

    # Extract categories from the original mapping
    categories = user_card_categories.get(card_name, [])
    category_label = ", ".join(categories) if categories else "general benefits"

    if not card_summaries:
        final_recommendations[card_name] = f"- ✅ **No better alternative found** for {card_name} in categories: {category_label}."
        continue

    # Choose best alternative if any
    recommendation = choose_best_card(category_label, card_summaries, [card_name])
    final_recommendations[card_name] = recommendation
    print(f"\n=== Recommendation for {card_name} ===\n{recommendation}\n")